In [3]:
import numpy as np
import pandas as pd
import re
import urllib
import json
import contextlib, unicodedata, sys
import matplotlib.pyplot as plt
import torch
import difflib
from difflib import SequenceMatcher
import os
from bs4 import B

In [4]:
def Diff(li1, li2):
    li_dif = [i for i in li1 + li2 if i not in li1 or i not in li2]
    return li_dif

In [5]:
def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

In [7]:
%cd /Users/quert/Downloads/gcp_tmp/raw_text
clusters = []
with open("title_page_whole_test.txt", "r") as fread:
    for line in fread.readlines():
        clusters.append(json.loads(line.strip()))

/Users/quert/Downloads/gcp_tmp/raw_text


In [9]:
clusters[0].keys()
# title_non_update_link
# title_update_link
# update_bs4
# non_update_bs4

dict_keys(['title', 'event_time', 'title_update_time', 'event_link', 'title_update_link', 'update_bs4', 'title_non_update_link', 'title_non_update_time', 'non_update_bs4', 'article_title', 'source_news_first', 'source_news_full', 'article_record', 'update_all_paragraph_with_references', 'update_first_paragraph', 'update_all_paragraph', 'non_update_all_paragraph_with_references', 'non_update_all_paragraph', 'non_update_first_paragraph', 'update_wiki_summary', 'non_update_wiki_summary', 'wiki_portal_summary', 'citation_position', 'event_wcep_id'])

### Get the indices of required instances
* Keeep English language, with non-updated full-content existed, and meaningful updated full-content

In [13]:
# create idx list
all_idx = [i for i in range(len(clusters))]
rmved_idx = []

for idx in range(len(clusters)):
    try:
        if bool(len(clusters[idx]['non_update_all_paragraph'].split('. ')) == 1):
            rmved_idx.append(idx)
    except:
        error_idx.append(idx)
idx_list = Diff(all_idx, rmved_idx)
len(all_idx), len(idx_list)

(239, 232)

In [26]:
rmved_idx = []
for idx in idx_list:
    if len(clusters[idx]['update_first_paragraph'].split())<=10:
        rmved_idx.append(idx)
id_list = Diff(idx_list, rmved_idx)
len(idx_list), len(id_list)

(232, 226)

In [27]:
# filter by language of summ==English
en_ids = []
for idx in id_list:
    if isEnglish(clusters[idx]['wiki_portal_summary']):
        en_ids.append(idx)
len(id_list), len(en_ids)
# after filtering, we have 201 instances in test set

(226, 201)

In [135]:
en_ids[:5]

[1, 2, 4, 5, 6]

### Parse the unicode into text
* Parse the unicode into text
* Paragraphs are separate with `. \\c\\c`

In [67]:
log = []
with open("test_text.txt.src", "w") as f:
    for idx in en_ids:
        bef_rec = []
        non_update_all_paragraph = clusters[idx]['non_update_all_paragraph'].split('\n')
        bef_rec.extend(non_update_all_paragraph)
        s = '.\c'
        bef_rec_str = s.join(bef_rec).replace("\\\\\c", "\c\c")
        with contextlib.redirect_stdout(fwrite):
            print({unicodedata.normalize('NFKD', bef_rec_str).encode('utf-8', 'ignore').decode('utf-8')})
with open("test_text.txt.src", "r") as f:
    for line in f.readlines():
        log.append(line.strip().replace(".\\\\c", "\c\c").replace("\\\'s", "'s").replace("{\'", "").replace("\'}", "").replace(".\\c\\c", ". \\c\\c"))
with open("test_text.txt.src", "w") as f:
    for line in log:
        f.write(line+"\n")

In [76]:
log = []
with open("test_text.txt.tgt", "w") as f:
    for idx in en_ids:
        tgt_rec = []
        update_all_paragraph = clusters[idx]["update_all_paragraph"].split("\n")
        tgt_rec.extend(update_all_paragraph)
        s = ".\c"
        tgt_rec_str = s.join(tgt_rec).replace("\\\\\c", "\c\c")
        with contextlib.redirect_stdout(f):
            print({unicodedata.normalize('NFKD', bef_rec_str).encode('utf-8', 'ignore').decode('utf-8')})
with open("test_text.txt.tgt", "r") as f:
    for line in f.readlines():
        log.append(line.strip().replace(".\\\\c", "\c\c").replace("\\\'s", "'s").replace("{\'", "").replace("\'}", "").replace(".\\c\\c", ". \\c\\c"))
with open("test_text.txt.tgt", "w") as f:
    for line in log:
        f.write(line+"\n")

### Extract the section name for each paragraph

In [93]:
text = clusters[0]["non_update_bs4"]

In [210]:
# section names except ("See also", "References")
text = clusters[0]["non_update_bs4"]
end_idx = re.findall('<span class="toctext">(.*)</span>', text).index('See also')
sec_names = re.findall('<span class="toctext">(.*)</span>', text)[:end_idx]
sec_names, len(sec_names)

(['Bidding process',
  'Host city selection',
  'Impact of the COVID-19 pandemic',
  'Qualifying event cancellation and postponement',
  'Effect on doping tests',
  'Postponement to 2021',
  'Calls for cancellation',
  'Costs and insurance',
  'Development and preparation',
  'Venues and infrastructure',
  'Security',
  'Volunteers',
  'Medals',
  'Torch relay',
  'Biosecurity protocols',
  'Ticketing',
  'Cultural festival',
  'The Games',
  'Opening ceremony',
  'Sports',
  'Test events',
  'Participating national Olympic committee teams',
  'Number of athletes by National Olympic Committee',
  'Medal summary',
  'Podium sweeps',
  'Calendar',
  'Event scheduling',
  'Marketing',
  'Victory ceremonies',
  'Colors',
  'Concerns and controversies',
  'Related to the Organizing Committee',
  'Broadcasting'],
 33)

In [225]:
# Split the numbers of paragraphs from each section
# section start from index 1, names start from index 0
'''
# clusters[0]["non_update_bs4"].split('<span class="mw-headline')[1].split("<p>")[1:]
# len(clusters[0]["non_update_bs4"].split('<span class="mw-headline'))-1
'''
'''
# num_pars = []
# for idx in range(len(sec_names)):
#     num_par = len(clusters[0]["non_update_bs4"].split('<span class="mw-headline')[idx+1].split("<p>")[1:])
#     num_pars.append(num_par)
# num_pars, len(num_pars)
'''
num_pars = [len(clusters[0]["non_update_bs4"].split('<span class="mw-headline')[i+1].split("<p>")[1:]) for i in range(len(sec_names))]
secs_pars = dict(zip(sec_names, num_pars))
# remove the section with no paragraphs (this problem ocurrs when the main section do not involved summary but only sub-sections)
secs_pars_nums = {k: v for k, v in secs_pars.items() if v!=0}
secs_pars_nums

{'Bidding process': 1,
 'Host city selection': 1,
 'Impact of the COVID-19 pandemic': 2,
 'Qualifying event cancellation and postponement': 1,
 'Effect on doping tests': 1,
 'Postponement to 2021': 3,
 'Calls for cancellation': 8,
 'Costs and insurance': 3,
 'Development and preparation': 5,
 'Venues and infrastructure': 4,
 'Security': 1,
 'Volunteers': 2,
 'Medals': 4,
 'Torch relay': 4,
 'Biosecurity protocols': 7,
 'Ticketing': 3,
 'Cultural festival': 2,
 'Opening ceremony': 1,
 'Sports': 8,
 'Test events': 3,
 'Participating national Olympic committee teams': 7,
 'Number of athletes by National Olympic Committee': 2,
 'Medal summary': 1,
 'Podium sweeps': 1,
 'Calendar': 3,
 'Event scheduling': 1,
 'Marketing': 2,
 'Victory ceremonies': 1,
 'Colors': 1,
 'Concerns and controversies': 11,
 'Related to the Organizing Committee': 8,
 'Broadcasting': 4}

* Since full-article includes "summary", we have to remove the "summary" before alignment -> then add back to the text afterwards


In [152]:
# summary
# summary = clusters[2]["non_update_wiki_summary"]
contents_with_summ = []
with open("test_text.txt.src", "r") as f:
    for line in f.readlines():
        contents_with_summ.append(line)

for idx in range(len(en_ids)):
    real_id = en_ids[idx]
    s = "\\c\\c"
    summs = clusters[real_id]["non_update_wiki_summary"].split("\n") # splitted summary
    full_content_with_summ = contents_with_summ[idx]
    with_summs = full_content_with_summ.split("\\c\\c") # splitted contents with summ
    for par in with_summs:
        if par in summs: with_summs.remove(par) 
with open("test_no_summs.txt.src", "w") as f:
    for line in with_summs: f.write(line+"\n")

In [153]:
no_summs = []
with open("test_no_summs.txt.src", "r") as f:
    for line in f.readlines(): no_summs.append(line)

In [154]:
no_summs[0]

"In March and April 2021, Russia started to mass thousands of military personnel and equipment near its border with Ukraine, representing the highest force mobilization since the country's annexation of Crimea in 2014. This precipitated an international crisis and generated concerns over a potential invasion. Satellite imagery showed movements of armor, missiles, and heavy weaponry. The troops were partially removed by June. The crisis was renewed in October and November 2021, when over 100,000 Russian troops were again massed near the border by December. \n"

In [ ]:
contents_with_summ[0]